In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
amazon = pd.read_csv('./amazon_cells_labelled.txt', sep='\t', header=None)
imdb = pd.read_csv('./imdb_labelled.txt', sep='\t', header=None)
yelp = pd.read_csv('./yelp_labelled.txt', sep='\t', header=None)

In [3]:
amazon

,0,1
0,So there is no way for me to plug it in here i...,0
1,"Good case, Excellent value.",1
2,Great for the jawbone.,1
3,Tied to charger for conversations lasting more...,0
4,The mic is great.,1
...,...,...
995,The screen does get smudged easily because it ...,0
996,What a piece of junk.. I lose more calls on th...,0
997,Item Does Not Match Picture.,0
998,The only thing that disappoint me is the infra...,0


In [4]:
np.count_nonzero(amazon[1])/amazon.shape[0]

0.5

In [5]:
yelp

,0,1
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1
...,...,...
995,I think food should have flavor and texture an...,0
996,Appetite instantly gone.,0
997,Overall I was not impressed and would not go b...,0
998,"The whole experience was underwhelming, and I ...",0


In [6]:
np.count_nonzero(yelp[1])/yelp.shape[0]

0.5

In [7]:
imdb

,0,1
0,"A very, very, very slow-moving, aimless movie ...",0
1,Not sure who was more lost - the flat characte...,0
2,Attempting artiness with black & white and cle...,0
3,Very little music or anything to speak of.,0
4,The best scene in the movie was when Gerardo i...,1
...,...,...
743,I just got bored watching Jessice Lange take h...,0
744,"Unfortunately, any virtue in this film's produ...",0
745,"In a word, it is embarrassing.",0
746,Exceptionally bad!,0


In [18]:
np.count_nonzero(imdb[1])/imdb.shape[0]

0.516042780748663

In [22]:
X_train_a, X_test_a, y_train_a, y_test_a = train_test_split(amazon[0], amazon[1])

In [63]:
joined_reviews = np.concatenate((amazon[0], imdb[0], yelp[0]))

In [67]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
am_vec = count_vect.fit_transform(joined_reviews)
X_train_counts.shape

(750, 1578)

In [53]:
X_train_a, X_test_a, y_train_a, y_test_a = train_test_split(am_vec, amazon[1])

In [54]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_a, y_train_a)

In [55]:
from sklearn.metrics import accuracy_score

In [59]:
#docs_new = ['Slow service, will not use again', 'Could not recommend enough', 'Will recommend', 'Will not recommend', 'Good', 'Great']
#X_new = count_vect.transform(X_test_a)
#X_new_tfidf = tfidf_transformer.transform(X_new_counts)

#predicted = clf.predict(X_new_tfidf)
predicted = clf.predict(X_test_a)
print(accuracy_score(y_test_a, predicted))
#sklearn.metrics.ac
#for doc, category in zip(docs_new, predicted):
#    print('%r => %s' % (doc, twenty_train.target_names[category]))

#'God is love' => soc.religion.christian
#'OpenGL on the GPU is fast' => comp.graphics

0.78


In [46]:
recenzja = ['Fast delivery', 'Good', 'Bad']
transformed = count_vect.transform(recenzja)
prediction = clf.predict(transformed)
prediction

array([1, 1, 0])